# Store kitti sequence

Create a sequence

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
#os.environ['STEREO_CONFIDENCE'] = '1.6'
#os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_0'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_1'

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 17.5 ms, sys: 3.21 ms, total: 20.7 ms
Wall time: 256 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [3]:
seq = Sequence()
for f in ProgressBar()(frame[:20]):
    seq.add_frame(f)

 80% (16 of 20) |####################     | Elapsed Time: 0:00:19 ETA:  0:00:05

invalid speed keyframe 10 frame 16 speed 4.312438936428539
 [[ 0.99965565  0.01930465  0.0177739  -1.12494169]
 [-0.01939701  0.99979916  0.00503886 -0.6222742 ]
 [-0.01767305 -0.00538188  0.99982933  0.18559034]
 [ 0.          0.          0.          1.        ]]


100% (20 of 20) |#########################| Elapsed Time: 0:00:24 Time: 0:00:24


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [8]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

cv_keyframes = get_covisible_keyframes(seq.keyframes[1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
print(len(cv_keyframes_BA), '\n')




# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


keyframes_np_MO = keyframes_to_np(seq.keyframes)

# np.save('cv_keyframes', cv_keyframes_BA)
# np.save('f_keyframes', f_keyframes_np)
# np.save('mappoints', mappoints_np)
# np.save('links', links_np)


Keyframes count
 4 

mappoints count  
 (75, '\n')
mappoints 
 [[  8.49000000e+02  -9.32748251e+00  -5.07361293e-01   3.25186756e+01
    1.00000000e+00]
 [  1.11100000e+03  -1.15003108e+01  -4.08782314e+00   1.00581406e+02
    1.00000000e+00]
 [  1.10800000e+03  -1.25009702e+01  -4.37509010e+00   1.00752499e+02
    1.00000000e+00]
 [  5.03000000e+02  -1.11848962e+01  -2.57116488e+00   5.24850366e+01
    1.00000000e+00]
 [  5.37000000e+02  -7.46427912e+00  -3.69356842e+00   2.00819102e+01
    1.00000000e+00]
 [  1.00100000e+03  -1.04741962e+01  -3.22676508e+00   1.80912278e+01
    1.00000000e+00]
 [  7.08000000e+02  -9.14977756e+00  -3.58923355e+00   3.47654482e+01
    1.00000000e+00]
 [  1.18400000e+03  -6.22454175e+00   1.27300818e+00   2.70868294e+01
    1.00000000e+00]
 [  4.11000000e+02  -1.17014685e+01  -8.89673918e-01   2.87880838e+01
    1.00000000e+00]
 [  7.29000000e+02  -7.19547474e+00  -3.30272873e+00   2.04291362e+01
    1.00000000e+00]
 [  3.70000000e+02  -2.54102628e+01  

In [4]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

cv_keyframes = get_covisible_keyframes(seq.keyframes[1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)

print(cv_keyframes)
poses = []
last = None
for i in range(len(cv_keyframes_np)):
    p = cv_keyframes_np[i]
    keyframeid = p[0]
    frameid = p[1]
    r = p[2:].reshape((4,4))
    if last is not None:
        r = np.dot(last, r)
        print(last.shape, r.shape, r)
        poses.append(np.hstack([keyframeid, frameid, r.reshape((16))]))
    else:
        poses.append(p)
    last = r

cv_keyframes_np = np.array(poses)

f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
print(len(cv_keyframes_BA), '\n')




# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


keyframes_np_MO = keyframes_to_np(seq.keyframes)

# np.save('cv_keyframes', cv_keyframes_BA)
# np.save('f_keyframes', f_keyframes_np)
# np.save('mappoints', mappoints_np)
# np.save('links', links_np)


Keyframes count
 4 

[<src.frame.Frame object at 0x7f2e8c585630>, <src.frame.Frame object at 0x7f2e8c5856d8>, <src.frame.Frame object at 0x7f2e8c585908>, <src.frame.Frame object at 0x7f2e8c585940>]
(4, 4) (4, 4) [[  9.99680592e-01   3.06030385e-03   2.50868282e-02  -1.60115496e-02]
 [ -3.22442870e-03   9.99973647e-01   6.50443292e-03   8.14327515e-03]
 [ -2.50662615e-02  -6.58324604e-03   9.99664115e-01  -4.30736314e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
(4, 4) (4, 4) [[  9.99119662e-01   1.02416990e-03   4.19386778e-02  -1.35080922e-01]
 [ -1.00057107e-03   9.99999329e-01  -5.83685083e-04   5.23501388e-02]
 [ -4.19392475e-02   5.41208615e-04   9.99120016e-01  -7.34232150e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
(4, 4) (4, 4) [[  9.98661618e-01   3.03789036e-03   5.16308417e-02  -2.83305177e-01]
 [ -3.02384532e-03   9.99995367e-01  -3.50139981e-04   1.24470936e-01]
 [ -5.16316662e-02   1.93547681e-04   9.98666177